In [8]:
import pandas as pd
from qbstyles import mpl_style
from datetime import datetime as dt

In [1]:
!mkdir "csv"

In [17]:
a = pd.date_range('01-01-2021', '01-03-2021')
# a.to_list()[0]
for i in a:
    b = i.strftime('%m-%d-%Y')
    !curl https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/01-01-2021.csv -o {"csv/" + b + ".csv"}
#     print(i.strftime('%m-%d-%Y'))
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9502  100  9502    0     0   114k      0 --:--:-- --:--:-- --:--:--  114k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9502  100  9502    0     0   356k      0 --:--:-- --:--:-- --:--:--  356k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9502  100  9502    0     0   343k      0 --:--:-- --:--:-- --:--:--  343k


In [2]:
!curl https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/01-01-2021.csv -o "csv/01-01-2021.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9502  100  9502    0     0   226k      0 --:--:-- --:--:-- --:--:--  226k
